In [10]:
from qrisp import jaspify, QuantumFloat, modinv, jasp_fourier_adder, gidney_adder, measure, boolean_simulation, QuantumModulus, QuantumBool, control
from qrisp.alg_primitives.arithmetic.jasp_arithmetic.jasp_montgomery import qq_montgomery_multiply, compute_aux_radix_exponent, cq_montgomery_multiply, best_montgomery_shift, cq_montgomery_multiply_inplace 
import numpy as np

In [2]:
@boolean_simulation
def qq(a, b, n, N):
    qa = QuantumFloat(n)
    qa[:] = a
    qb = QuantumFloat(n)
    qb[:] = b
    m = compute_aux_radix_exponent(N, n)
    res = qq_montgomery_multiply(qa, qb, N, m, gidney_adder)
    return measure(qa), measure(qb), measure(res) 

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    q = modinv(2**n, N)
    for a in range(1, 50, 3):
        for b in range(1, 50, 5):
            if a%N != 0 and b%N != 0:
                ar, br, rr = qq(a%N, b%N, n, N)
                assert ar == a%N
                assert br == b%N
                assert rr == (ar*br*q)%N

In [3]:
@boolean_simulation
def cq(a, b, n, N):
    qb = QuantumFloat(n)
    qb[:] = b
    shift = best_montgomery_shift(a, N)
    res = cq_montgomery_multiply(a, qb, N, shift, gidney_adder)
    return measure(qb), measure(res) 

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    q = modinv(2**n, N)
    for a in range(4, 50, 3):
        for b in range(4, 50, 5):
            if a%N != 0 and b%N != 0:
                br, rr = cq(a%N, b%N, n, N)
                assert br == b%N
                assert rr == ((a%N)*br)%N

In [6]:
@boolean_simulation
def icq(a, b, n, N):
    qb = QuantumFloat(n)
    qb[:] = b
    shift = best_montgomery_shift(a, N)
    cq_montgomery_multiply_inplace(a, qb, N, shift, gidney_adder)
    return measure(qb)

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    q = modinv(2**n, N)
    for a in range(4, 50, 3):
        for b in range(4, 50, 5):
            if a%N != 0 and b%N != 0 and np.gcd(a, N) == 1:
                br = icq(a%N, b%N, n, N)
                assert br == ((a%N)*(b%N))%N

In [11]:
@boolean_simulation
def cicq(a, b, n, N, c):
    qb = QuantumFloat(n)
    qb[:] = b
    shift = best_montgomery_shift(a, N)
    qc = QuantumBool()
    qc[:] = c
    with control(qc[0]):
        cq_montgomery_multiply_inplace(a, qb, N, shift, gidney_adder)
    return measure(qb)

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    q = modinv(2**n, N)
    for a in range(4, 50, 3):
        for b in range(4, 50, 5):
            for c in [0, 1]:
                if a%N != 0 and b%N != 0 and np.gcd(a, N) == 1:
                    br = cicq(a%N, b%N, n, N, c)
                    assert br == (((a%N)**c)*(b%N))%N

In [171]:
import numpy as np
from qrisp import boolean_simulation, QuantumFloat, QuantumBool, modinv, gidney_adder, measure, control, BigInteger
from qrisp.alg_primitives.arithmetic.jasp_arithmetic.jasp_montgomery import cq_montgomery_multiply_inplace, best_montgomery_shift
@boolean_simulation
def bicicq(a, b, n, N, c):
    a = BigInteger.from_int(a, 3)
    N = BigInteger.from_int(N, 3)
    qb = QuantumFloat(n)
    qb[:] = b
    shift = best_montgomery_shift(a, N)
    qc = QuantumBool()
    qc[:] = c
    with control(qc[0]):
        cq_montgomery_multiply_inplace(a, qb, N, shift, gidney_adder)
    return measure(qb)

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    for a in range(4, 50, 3):
        for b in range(4, 50, 5):
            for c in [0, 1]:
                if a%N != 0 and b%N != 0 and np.gcd(a, N) == 1:
                    br = bicicq(a%N, b%N, n, N, c)
                    assert br == (((a%N)**c)*(b%N))%N

In [67]:
from qrisp import QuantumFloat, modinv, gidney_adder, multi_measurement
from qrisp.alg_primitives.arithmetic.jasp_arithmetic.jasp_montgomery import qq_montgomery_multiply, compute_aux_radix_exponent

X = 29
y = 21
N = 31
n = 5

def test_qq_g():
    qx = QuantumFloat(n)
    qx[:] = X
    qy = QuantumFloat(n)
    qy[:] = y
    m = compute_aux_radix_exponent(N, n)
    res = qq_montgomery_multiply(qx, qy, N, m, gidney_adder)
    r = QuantumFloat(n)
    return multi_measurement([qx])

m = compute_aux_radix_exponent(N, n)

r = ((X*y*modinv(2**m, N))%N,)
test_qq_g()

AttributeError: 'int' object has no attribute 'identifier'

In [58]:
@boolean_simulation
def qq(a, b, n, N):
    qa = QuantumModulus(13)
    #qa.m = 4
    #qa[:] = 3
    qb = QuantumModulus(13)
    #qb.m = 4
    #qb[:] = 3
    #res = qa * qb
    return measure(qa), measure(qb)#, measure(res) 

for N in range(11, 50, 8):
    n = int(np.ceil(np.log2(N)))
    q = modinv(2**n, N)
    for a in range(1, 50, 3):
        for b in range(1, 50, 5):
            if a%N != 0 and b%N != 0:
                qq(a%N, b%N, n, N)
                continue
                ar, br, rr = qq(a%N, b%N, n, N)
                assert ar == a%N
                assert br == b%N
                assert rr == (ar*br*q)%N

TypeError: lax.rem requires arguments to have the same dtypes, got float64, int64.